<a href="https://colab.research.google.com/github/s-c-soma/AdvanceDeeplearning-CMPE-297/blob/master/Assignment_3/ExtraCredit_Assignment_3_Part_D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3 Part D

## Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tqdm import tqdm

## SIMCLR

In [ ]:
class simclr(object):
    def __init__(self):
        self.batch_size = 64

    def resnet_simclr(self, layer1, layer2, layer3):
        model = tf.keras.applications.ResNet50(include_top=False, weights=None, input_shape=(224,224,3))
        model.trainable = True
        inputs = Input((224,224,3))
        h = model(inputs, trainable=True)
        h = GlobalAveragePooling2D()(h)

        layer1_projection = Dense(layer1)(h)
        layer1_activation = Activation("relu")(layer1_projection)
        layer2_projection = Dense(layer2)(layer1_activation)
        layer2_activation = Activation("relu")(layer2_projection)
        layer3_projection = Dense(layer3)(layer2_activation)

        resnet_simclr_model = Model(inputs, layer3_projection)

        return resnet_simclr_model

    def train_step(self, input1, input2, model, optimizer, criterion, temperature):
        with tf.GradientTape() as tape:
            norm_input1 = model(input1)
            norm_input1 = tf.math.l2_normalize(norm_input1, axis=1)
            norm_input2 = model(input2)
            norm_input2 = tf.math.l2_normalize(norm_input2, axis=1)

            simililarity_pos = _dot_simililarity_dim1(norm_input1, norm_input2)
            simililarity_pos = tf.reshape(simililarity_pos, (self.batch_size, 1))
            simililarity_pos /= temperature

            negatives = tf.concat([norm_input1, norm_input2], axis=0)

            loss = 0

            for positives in [norm_input1, norm_input2]:
                simililarity_neg = _dot_simililarity_dim2(positives, negatives)

                labels = tf.zeros(self.batch_size, dtype=tf.int32)

                negative_mask = get_negative_mask(self.batch_size)
                simililarity_neg = tf.boolean_mask(simililarity_neg, negative_mask)
                simililarity_neg = tf.reshape(simililarity_neg, (self.batch_size, -1))
                simililarity_neg /= temperature

                logits = tf.concat([simililarity_pos, simililarity_neg])
                loss += criterion(logits, labels)

            loss /= 2 * self.batch_size
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        return loss

In [ ]:
def _dot_simililarity_dim1(x, y):
    v = tf.matmul(tf.expand_dims(x, 1), tf.expand_dims(y, 2))
    return v

In [ ]:
def _dot_simililarity_dim2(x, y):
    v = tf.tensordot(tf.expand_dims(x, 1), tf.expand_dims(tf.transpose(y), 0), axes=2)
    return v

In [ ]:
def get_negative_mask(batch_size):
    negative_mask = np.ones((batch_size, 2 * batch_size), dtype=bool)
    for i in range(batch_size):
        negative_mask[i, i] = 0
        negative_mask[i, i + batch_size] = 0
    return tf.constant(negative_mask)

In [ ]:
if __name__ == '__main__':
    print(tf.__version__)
    simclr1=simclr()

2.3.0
